In [1]:
import pandas as pd
import os

In [215]:
vcf_file = [file for file in os.listdir(os.getcwd()) if file.startswith('merge_output')]

In [216]:
vcf_file

['merge_output_40.vcf']

In [207]:
def vcf_filter(vcf_file,ref_fasta):
    with open(vcf_file, 'r') as f:
        content = f.read()
        lines = content.splitlines()
        
    vcf_metadata = '\n'.join(lines[0:17])
    
    df = pd.read_csv(vcf_file, sep = '\t', 
                     comment = '#',
                     header = None)
    df.columns = ['VirusName','Position',
                  'ID','Ref','Alt','Quality',
                  'Filter','info','format',
                  'sample']
    df['Genotype'] = df['sample'].apply(lambda x:float(x.split(':')[0]))
    df['GenotypeQuality'] =df['sample'].apply(lambda x:float(x.split(':')[1]))
    df['Depth'] = df['sample'].apply(lambda x:float(x.split(':')[2]))
    pass_vcf = df[(df['GenotypeQuality'] > 3) & (df['Depth'] > 20)]
    fail_vcf = df[(df['GenotypeQuality'] < 3) | (df['Depth'] < 20)]
    with open(ref_fasta, 'r') as f:
        content = f.read()
        
    lines = content.splitlines()

    IDs = []
    seqs = []
    current_id = None
    current_seq = []

    for line in lines:
        if line.startswith('>'):
        
            if current_id is not None:
                IDs.append(current_id)
                seqs.append(''.join(current_seq))

            current_id = line[1:].strip()
            current_seq = []
        else:
            current_seq.append(line)

    if current_id is not None:
        IDs.append(current_id)
        seqs.append(''.join(current_seq))
    nbase = []
    position = []
    for i in range(len(seqs[0])):
        base = seqs[0][i]
        pos = i + 1
        nbase.append(base)
        position.append(pos)
    nbase = pd.DataFrame({'ref':nbase})
    position = pd.DataFrame({'pos':position})
    df = pd.concat([position,nbase],axis =1)
    ref_n_pos = []
    for pos in fail_vcf['Position']:
        df_pos = df[df['pos'] == pos]
        ref_n_pos.append(df_pos)
    ref_base_pos = []
    for pos in pass_vcf['Position']:
        df_pos = df[df['pos'] == pos]
        ref_base_pos.append(df_pos)
    fail_ref_fasta = pd.concat(ref_n_pos)
    pass_ref_fasta = pd.concat(ref_base_pos)
    nucle_to_be_replaced = fail_ref_fasta['ref'].value_counts().keys()
    
    fail_ref_fasta = fail_ref_fasta.replace(nucle_to_be_replaced,'N')
    manipulated_ref_fasta = pd.concat([pass_ref_fasta,fail_ref_fasta])#.reset_index().drop(['index'],axis =1 ).sort_values('pos')
    ref_pos = set(df['pos'])
    man_pos = set(manipulated_ref_fasta['pos'])
    tmp = ref_pos - man_pos
    pos_ref_tmp = []
    for pos in tmp:
        pos_tmp = df[df['pos'] == pos]
        pos_ref_tmp.append(pos_tmp)
    pos_ref_tmp = pd.concat(pos_ref_tmp)
    manipulated_ref = pd.concat([manipulated_ref_fasta,pos_ref_tmp]).sort_values('pos')
    fasta_df = pd.DataFrame()
    fasta_df['ID']  = IDs
    fasta_df['Seq']  = ''.join(list(manipulated_ref['ref']))
    fasta_name = 'barcode' + vcf_file.split('.')[0].split('_')[-1] + '_consensus.fasta'
    with open(fasta_name, 'w') as fasta_file:
       for index, row in fasta_df.iterrows():
           fasta_file.write(f">{row['ID']}\n{row['Seq']}\n")
    pass_vcf = pass_vcf[['VirusName','Position','ID','Ref','Alt','Quality','Filter','info','format','sample']]
    pass_vcf = pass_vcf.rename(columns= {'VirusName':'CHROM','Position':'POS',
                              'Ref':'REF','Alt':'ALT','Quality':'QUAL',
                              'Filter':'FILTER','info':'INFO','format':'FORMAT','sample':'SAMPLE'})
    fail_vcf = fail_vcf[['VirusName','Position','ID','Ref','Alt','Quality','Filter','info','format','sample']]
    fail_vcf = fail_vcf.rename(columns= {'VirusName':'CHROM','Position':'POS',
                              'Ref':'REF','Alt':'ALT','Quality':'QUAL',
                              'Filter':'FILTER','info':'INFO','format':'FORMAT','sample':'SAMPLE'})
    fail_vcf = fail_vcf.to_string(index=False)
    pass_vcf = pass_vcf.to_string(index=False)
    combined_content_fail = vcf_metadata + "\n\n" + fail_vcf
    combined_content_pass = vcf_metadata + "\n\n" + pass_vcf
    pass_file_name = 'barcode' + vcf_file.split('.')[0].split('_')[-1] + '_pass' 
    fail_file_name = 'barcode' + vcf_file.split('.')[0].split('_')[-1] + '_fail' 
    with open(pass_file_name + ".vcf", "w") as file:
        file.write(combined_content_pass)
    with open(fail_file_name + ".vcf", "w") as file:
        file.write(combined_content_fail)
    
    
    
    return combined_content_fail





In [225]:
vcf_filter('merge_output_40.vcf','ref.fasta')

In [214]:
for file in vcf_file[1:]:
    vcf_filter(file,'ref.fasta')
    print(file)
    

merge_output_41.vcf
merge_output_5.vcf
merge_output_6.vcf
merge_output_7.vcf
merge_output_8.vcf
merge_output_9.vcf


'barcode1_consensus_fasta'

In [190]:
#import pandas as pd
#
## Load the text file
#with open("text_file.txt", "r") as file:
#    text_content = file.read()

# Assuming df is your DataFrame
# Example DataFrame creation
data = {'Name': ['Alice', 'Bob', 'Charlie'],
        'Age': [25, 30, 35],
        'City': ['New York', 'Los Angeles', 'Chicago']}
df = pd.DataFrame(data)

# Convert DataFrame to string
df_string = df




In [191]:
df_string

'   Name  Age        City\n  Alice   25    New York\n    Bob   30 Los Angeles\nCharlie   35     Chicago'

In [ ]:
# Combine text content and DataFrame content
combined_content = text_content + "\n\n" + df_string

# Save the combined content to a new text file
with open("combined_file.txt", "w") as file:
    file.write(combined_content)

print("Combined file saved successfully.")

In [180]:
with open('merge_output_2.vcf', 'r') as f:
    content = f.read()
    lines = content.splitlines()
        

In [181]:
lines[0:17]

['##fileformat=VCFv4.2',
 '##source=Clair3',
 '##clair3_version=1.0.4',
 '##cmdline=/hpcigib/varsha.r/miniconda3/envs/clair3/bin/run_clair3.sh --bam_fn barcode02.trimmed.rg.sorted.bam --ref_fn /hpcigib/varsha.r/tools/artic-ncov2019/primer_schemes/denv2/V1/denv2.reference.fasta --threads=10 --platform ont --model_path /hpcigib/varsha.r/tools/Clair3/models/r941_prom_sup_g5014 --output clair_VCF/barcode02 --no_phasing_for_fa --include_all_ctgs --haploid_precise --var_pct_full=0.8',
 '##reference=/hpcigib/varsha.r/tools/artic-ncov2019/primer_schemes/denv2/V1/denv2.reference.fasta',
 '##FILTER=<ID=PASS,Description="All filters passed">',
 '##FILTER=<ID=LowQual,Description="Low quality variant">',
 '##FILTER=<ID=RefCall,Description="Reference call">',
 '##INFO=<ID=P,Number=0,Type=Flag,Description="Result from pileup calling">',
 '##INFO=<ID=F,Number=0,Type=Flag,Description="Result from full-alignment calling">',
 '##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">',
 '##FORMAT=<ID=

In [182]:
'\n'.join(lines[0:17])

'##fileformat=VCFv4.2\n##source=Clair3\n##clair3_version=1.0.4\n##cmdline=/hpcigib/varsha.r/miniconda3/envs/clair3/bin/run_clair3.sh --bam_fn barcode02.trimmed.rg.sorted.bam --ref_fn /hpcigib/varsha.r/tools/artic-ncov2019/primer_schemes/denv2/V1/denv2.reference.fasta --threads=10 --platform ont --model_path /hpcigib/varsha.r/tools/Clair3/models/r941_prom_sup_g5014 --output clair_VCF/barcode02 --no_phasing_for_fa --include_all_ctgs --haploid_precise --var_pct_full=0.8\n##reference=/hpcigib/varsha.r/tools/artic-ncov2019/primer_schemes/denv2/V1/denv2.reference.fasta\n##FILTER=<ID=PASS,Description="All filters passed">\n##FILTER=<ID=LowQual,Description="Low quality variant">\n##FILTER=<ID=RefCall,Description="Reference call">\n##INFO=<ID=P,Number=0,Type=Flag,Description="Result from pileup calling">\n##INFO=<ID=F,Number=0,Type=Flag,Description="Result from full-alignment calling">\n##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">\n##FORMAT=<ID=GQ,Number=1,Type=Integer,Descripti